# Orbit Simulator Tutorial

The FSSC Orbit Simulator, **gtorbsim**, is a spacecraft attitude calculator. It has a number of capabilities based on the code already implemented in the general purpose scheduling and planning system TAKO (Timeline Assembler Keyword Oriented) at the Fermi Science Support Center.

It is anticipated, however, that the typical end user will only need a subset of the overall functionality of this tool. Primarily, a Guest Investigator would use it to generate a spacecraft data file to use in conjunction with gtobssim to simulate Fermi-LAT observations.

This tutorial provides some examples of how to run the **gtorbsim** application. This tool generates a spacecraft data file that is an input for some of the Fermitools. For example, the user may need to run this tool before running gtobssim in order to generate a simulated observation.

The main purpose of this simulator is:

1. To calculate spacecraft attitude, that is where the local body frame axes are oriented relative to the sky

2. To determine when events such as entry/exit in South Atlantic Anomaly (SAA) will take place

The above must be accomplished starting with a series of pointing commands. The output of the orbit simulator is a FITS spacecraft data file.

You may use the spacecraft data file provided by the FSSC, but in many cases you will probably need to generate that file if you want to perform a particular analysis of simulated data.

» [Fermitools References](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/references.html)

Steps

1. **Observation Modes**

    Observation Modes available in gtorbsim.


2. **Spacecraft Ephemeris**

    Ephemeris files that gtorbsim can handle.


3. **Initial spacecraft position**
4. **The South Atlantic Anomaly region**
5. **Earth Limb**
6. **Run gtorbsim**

# 1. Observation Modes

The first input you will have to provide to gtorbsim is the observation mode strategy. Several operational modes are planned for Fermi, but the spacecraft will acquire scientific data only in survey and pointed modes.

The **sky survey mode** is basically zenith pointed throughout the orbit and has two sub modes: 1) with rocking, and 2) without rocking. Rocking provides more uniform sky coverage and allows for complete sky coverage within a shorter period of time. Different rocking profiles may be implemented, (square or sinusoid) with a basic 2-orbit period and a 60-degree maximum amplitude (above and below the orbit plane).

In **pointed observation mode**, the Z-axis of the observatory is commanded to point at a celestial target.

With the **gtorbsim** tool you may choose to calculate the attitude in survey or pointed mode. In survey mode you may choose between two options: fixed or profiled. 

In fixed survey mode the spacecraft does a sky survey with a specified offset with respect to its local zenith for one orbit, and then uses the opposite offset for the next orbit, and so on. In profiled survey, the spacecraft observes in survey mode according to a specified profile consisting of 17 increasing times and 17 zenith offsets. The 17 increasing times (in seconds) are used to indicate the time that it takes during each cycle to go from a corresponding zenith offset to the next. The 17 angles (in degrees) are the zenith offsets reached at the end of the corresponding time interval. The first and last of these offsets must be identical in order for the profile to be repeated.

On the other hand, in pointed mode the spacecraft stares at a specified location in the sky identified by an RA and DEC provided by the user. 

# 2. Spacecraft Ephemeris

 The orbit simulator needs to know the spacecraft position in the entire interval of interest in order to properly calculate the attitude, therefore it must be capable of either reading in a file that contains the spacecraft ephemeris, or to calculate one on the fly. The orbit simulator can handle three different types of ephemeris files:

1. NASA Flight Dynamic Facility (FDF) format, used for missions such as RXTE

2. Satellite Tool Kit (STK) format (also in use for Swift)

3. [NORAD Two Line Elements](http://celestrak.com/NORAD/elements) available online in which case the spacecraft position is calculated on the fly. The user may extract the Fermi TLE into a file from this [website](http://celestrak.com/NORAD/elements/science.txt). An example of this type of file is given below:
```
    FGRST (GLAST)
    1 33053U 08029A 09033.78481577 .00000278 00000-0 00000+0 0 1746
    2 33053 25.5831 172.9725 0014128 40.2809 319.8772 15.04902884 35615
```

**Notes**:

1. For the tool to work the first line of this file has to be "GLAST" and not "Fermi".

2. Keep in mind that the Two Line Elements files are valid only for a few days. If the spacecraft data file that you may want to create is for a period of time where the ephemeris is not valid your spacecraft data file may not be accurate.

3. You should copy and paste the example above into a file for use later in the example.

# 3. Initial Spacecraft position

The initial spacecraft position in equatorial coordinates should be provided by the user as an input parameter. 

# 4. The South Atlantic Anomaly region

The instrument high voltage power supplies will be protected when the spacecraft traverses the South Atlantic Anomaly (SAA). This will occur about ~15% of the time.

Gtorbsim has the capability to handle SAA constrains. The SAA region is appoximated by a polygon, which is specified by the Longitude and Latitude of its vertices. It is passed to the program as an input file where the specification of the polygon is given. In cases where the file is not available, a default hard-coded table of longitude and latitude pairs of vertices will be used.

The SAA file that is currently used can be downloaded from here.

In [ ]:
!wget https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/OrbSim/L_SAA_2008198.03

# 5. Earth Limb

The Earth Limb Tracing maneuvering is an optional feature that can easily be enabled/disabled using the appropriate input parameter in gtorbsim. This maneuvering consists of tracing the Earth Limb if a target is Earth-occulted.

Targets are assumed to be occulted if their Earth angle (angle between target and the Earth limb) is smaller or at most equal to 30 degrees. Once the target is occulted by the Earth, the orbit simulator finds when is visible again, and where from the Earth Limb is coming out. From the above it finds the angular separation between the in-occult and out-occult position. Finally, the orbit simulator let the local z-axis to sweep equal angles in equal times during its motion along the Earth Limb. 

# 6. Run gtorbsim

This section shows different ways to generate **survey mode** spacecraft data file. To generate a realistic pointed mode observation a timeline generated by TAKO is needed. The FSSC does not provide the user with that tool.

**If you need to run a realistic pointed mode observation, please contact the [FSSC](fermihelp@milkyway.gsfc.nasa.gov).**

There are different ways to enter the parameters in the tool: By answering a prompt or as a list in a command line, or using an input file. For this reason, the very first input of the simulator is the type of input, which can either be "console" or "file".

An example of init file is given below: 

```
start_MJD = 54867
stop_MJD = 54923
TLType = SINGLE
Timeline = |SURVEY|+50.0|
EphemFunc = tlederive
EphemName = FERMI_TLE_09033.78481577.tle
Units = 1.0
Resolution = 1
Initial_RA = 0
Initial_DEC = 0
saafunc = saa
saafile = L_SAA_2008198.03
OutPutFile = spacecraft_data_file_FERMI_TLE_09033.78481577.fits
EAA = 20
```

For the list of inputs refer to the [gtorbsim](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtorbsim.txt) help file.

If the inputs are given in a file you can run the tool like this: 

```
prompt> gtorbsim
Type of input {file or console}[file]
Input Type is: file
Name of init file[] spacecraft_data_file_FERMI_TLE_09033.78481577.init
Earth Avoidance Angle: 20 degrees
No target found in occultation
```

Otherwise you can run the tool in this way: 

```
prompt> gtorbsim EAA=20
Type of input {file or console}[file] console
Input Type is: console
start MJD[] 54867
stop MJD[] 54923
Timeline Type {TAKO, ASFLOWN or SINGLE}[TAKO] SINGLE
Timeline SINGLE Command[|SURVEY| +35.0 |]
Ephemeredis file name[] FERMI_TLE_09033.78481577.tle
Ephemeredis function name[xyzll_eph] tlederive
Conversion factor to Km[1]
Time resolution in minutes[1]
Initial RA[] 0
Initial DEC[] 0
OutPut File[] spacecraft_data_file_FERMI_TLE_09033.78481577.fits
SAA file definition[] L_SAA_2008198.03
Earth Avoidance Angle: 20 degrees
No target found in occultation
```

The spacecraft data file generated with this example could be downloaded from [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/OrbSim/spacecraft_data_file_FERMI_TLE_09033.78481577.fits). Keep in mind that this spacecraft data file is accurate only for the first few days.

Another spacecraft file generated with different ephemeris could be downloaded from [here](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/data/OrbSim/spacecraft_data_file_FERMI_EPH_2009012_2009068_00.fits).

# Related Tools

* [gtobssim](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtobssim.txt)